In [94]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [95]:

service = Service()

options = webdriver.ChromeOptions()

nav = webdriver.Chrome(service=service, options=options)

nav.maximize_window()

url = "https://books.toscrape.com/"

nav.get(url)

In [96]:
list_title = []
list_prix = []
stock = []

In [97]:
# Boucle pour scraper les livres
while True:
    # Récupérer les éléments sur la page actuelle
    list_element = nav.find_elements(By.CLASS_NAME, 'product_pod')  # Récupérer les livres

    # Scraper les informations pour chaque livre sur la page
    for book in list_element:
        title = book.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a').get_attribute('title')
        list_title.append(title)

        # Cliquer sur le livre pour accéder à la page de détails
        book.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a').click()
        
        # Scraper le prix
        prix = nav.find_element(By.CLASS_NAME, 'price_color').text.replace("£", "R$")
        list_prix.append(prix)

        # Scraper la disponibilité en stock
        val_stock = int(nav.find_element(By.CLASS_NAME, 'instock').text.replace("In stock (", "").replace(" available)", ""))
        stock.append(val_stock)

        # Retourner à la page des livres
        nav.back()

    # Essayer de trouver le bouton "Next" pour passer à la page suivante
    try:
        next_button = WebDriverWait(nav, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, 'next')))
        next_button.click()  # Cliquez sur le bouton "Next"
    except:
        print("Plus de pages à scraper.")
        break  # Quitter la boucle si le bouton "Next" n'est pas trouvé
    # Créer un DataFrame avec les données collectées
if len(list_title) == len(list_prix) == len(stock):
    books_df = pd.DataFrame({
        "Livres": list_title,
        "Prix": list_prix,
        "Stock": stock
    })
    
    books_df.index += 1

Plus de pages à scraper.


In [98]:
# Afficher le DataFrame
print(books_df)

                                                 Livres     Prix  Stock
1                                  A Light in the Attic  R$51.77     22
2                                    Tipping the Velvet  R$53.74     20
3                                            Soumission  R$50.10     20
4                                         Sharp Objects  R$47.82     20
5                 Sapiens: A Brief History of Humankind  R$54.23     20
...                                                 ...      ...    ...
996   Alice in Wonderland (Alice's Adventures in Won...  R$55.53      1
997    Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)  R$57.06      1
998   A Spy's Devotion (The Regency Spies of London #1)  R$16.97      1
999                 1st to Die (Women's Murder Club #1)  R$53.98      1
1000                 1,000 Places to See Before You Die  R$26.08      1

[1000 rows x 3 columns]


In [99]:
# Enregistrer le DataFrame dans un fichier CSV
books_df.to_csv('books_scraped.csv', index_label='Index', encoding='utf-8')

# Fermer le navigateur
nav.quit()